In [1]:
# --- 1. CORE LIBRARIES AND UTILITIES ---
# Standard data manipulation and visualization libraries
import numpy as np
import pandas as pd
from time import time

# --- 2. SCIKIT-LEARN: MODEL SELECTION & METRICS ---
# Tools for splitting data, cross-validation, and performance evaluation
from sklearn.model_selection import (
    train_test_split, 
    KFold, 
    cross_val_score, 
    GridSearchCV
)
from sklearn.metrics import (
    classification_report, 
    confusion_matrix, 
    accuracy_score, 
    roc_auc_score
)
from sklearn.pipeline import Pipeline # For creating robust modeling workflows
from sklearn.impute import SimpleImputer # Tool to handle NaNs
from sklearn.preprocessing import FunctionTransformer, StandardScaler

import lightgbm as lgb

import os
import sys

In [2]:
# Define project Path in Colab
PROJECT_BASE_PATH = '/content/drive/MyDrive/Project_01' 

# ADD 'src' DIRECTORY TO PYTHON PATH
SRC_PATH = os.path.join(PROJECT_BASE_PATH, 'src')

# verify if SRC_PATH is already in sys.path
if SRC_PATH not in sys.path:
    sys.path.insert(0, SRC_PATH)
    print("✅ Successfully added 'src' directory to Python path.")

# IMPORT Paths CLASS FROM config MODULE
from config import Paths

✅ Successfully added 'src' directory to Python path.


In [3]:
try:
    from config import Paths
    
    # 3. Inicialize a instância com um nome único (cfg)
    cfg = Paths(PROJECT_BASE_PATH) # <-- Mudança aqui
    cfg.create_dirs() 
    
    print("\n✅ Project configuration (Paths) initialized successfully.")
    print(f"Raw Data Path check: {cfg.TRAIN_RAW_FILE}")
    
except ImportError:
    print("❌ Error: Could not import Paths from config module.")


✅ Project configuration (Paths) initialized successfully.
Raw Data Path check: /content/drive/MyDrive/Project_01/data/raw/application_train.csv


In [4]:
FINAL_TRAIN_FILE = os.path.join(cfg.DATA_PROCESSED_DIR, 'train_final_encoded.parquet')
FINAL_TEST_FILE = os.path.join(cfg.DATA_PROCESSED_DIR, 'test_final_encoded.parquet')

try:
    # Read the Parquet files
    df_train_final = pd.read_parquet(FINAL_TRAIN_FILE)
    df_test_final = pd.read_parquet(FINAL_TEST_FILE)
    
    print(f"✅ Loaded Encoded Train Data. Shape: {df_train_final.shape}")
    print(f"✅ Loaded Encoded Test Data. Shape: {df_test_final.shape}")

except FileNotFoundError:
    print("❌ ERROR: Parquet files not found. Ensure Block 13 was executed successfully and saved the files.")
    # Exit or raise an error if critical data is missing
    raise

✅ Loaded Encoded Train Data. Shape: (307511, 135)
✅ Loaded Encoded Test Data. Shape: (48744, 134)


In [5]:
print("--- Defining bases and universal Imputer ---")

# 1. Prepare Features (X) and Target (y)
EXCLUDED_COLS = ['SK_ID_CURR', 'TARGET'] 
features = [col for col in df_train_final.columns if col not in EXCLUDED_COLS]

X = df_train_final[features]
y = df_train_final['TARGET']

--- Defining bases and universal Imputer ---


In [ ]:
# Hyperparameter Tuning via Grid Search (LightGBM)

print("--- Starting: Grid Search for LightGBM ---")

# 1. Prepare Data

# 2. Define the Model
lgbm_estimator = lgb.LGBMClassifier(
    objective='binary',
    metric='auc',
    boosting_type='gbdt',
    n_jobs=-1,
    random_state=42,
    verbose=-1
)

# 3. Define the Hyperparameter Grid
# We keep the grid small to manage Colab's time/memory limits.
param_grid = {
    # Number of trees: Test low and high values
    'n_estimators': [500, 1000], 
    
    # Learning rate (step size): Crucial for speed vs. accuracy
    'learning_rate': [0.05, 0.01],
    
    # Tree complexity (affects overfitting): Test low, medium, and high complexity
    'num_leaves': [15, 31, 63],
}

# 4. Set up Grid Search with Cross-Validation
# Use 3 folds (NFOLDS=3) for speed. Scoring is based on AUC.
NFOLDS = 3
kfold = KFold(n_splits=NFOLDS, shuffle=True, random_state=42)

grid_search = GridSearchCV(
    estimator=lgbm_estimator,
    param_grid=param_grid,
    scoring='roc_auc',
    cv=kfold,
    n_jobs=-1,
    verbose=2  # Show detailed progress
)

# 5. Execute Grid Search
print(f"--- Grid Search running on {len(param_grid['n_estimators']) * len(param_grid['learning_rate']) * len(param_grid['num_leaves'])} combinations ---")
print(f"Total fits to perform: {NFOLDS * len(param_grid['n_estimators']) * len(param_grid['learning_rate']) * len(param_grid['num_leaves'])}")

start_time = time()
grid_search.fit(X, y)
end_time = time()

print(f"✅ Grid Search completed in {(end_time - start_time):.2f} seconds.")


# 6. Extract and Report Results

# Get the best parameters and score
best_params = grid_search.best_params_
best_score = grid_search.best_score_
best_estimator = grid_search.best_estimator_

print("\n=======================================================")
print(f"🏆 Optimal Hyperparameters Found via Grid Search:")
print(f"Best AUC Score (Mean CV): {best_score:.4f}")
print(f"Best Parameters: {best_params}")
print("=======================================================")


# 7. Final Model for Deployment

# Update the global model variable with the optimized settings for Block 15 reuse
model_optimized_lgbm = best_estimator 
print("\nModel optimized with best parameters stored in 'model_optimized_lgbm'.")

--- Starting: Grid Search for LightGBM ---
--- Grid Search running on 12 combinations ---
Total fits to perform: 36
Fitting 3 folds for each of 12 candidates, totalling 36 fits
